## **Setup: Setting up the SQLite DB:**

In [105]:
import sqlite3
import pandas as pd

In [106]:
# Open up a new connection:
conn = sqlite3.connect('rpg_db.sqlite3')

In [107]:
# Open new cursor:
curs = conn.cursor()

## **Assignment - Part 1, Querying a Database:**

This directory contains a file rpg_db.sqlite3, a database for a hypothetical webapp role-playing game. This test data has dozens-to-hundreds of randomly generated characters across the base classes (Fighter, Mage, Cleric, and Thief) as well as a few Necromancers. Also generated are Items, Weapons, and connections from characters to them. Note that, while the name field was randomized, the numeric and boolean fields were left as defaults.

Use sqlite3 to load and write queries to explore the data, and answer the following questions:

**(1) How many total Characters are there?**

In [108]:
num_chars = curs.execute("SELECT COUNT(DISTINCT character_id) FROM charactercreator_character").fetchall()[0][0]
print(f"(1) How many total Characters are there?: {num_chars} Characters")

(1) How many total Characters are there?: 302 Characters


**(2) How many of each specific subclass?**

In [109]:
# SQLite queries to find how many unique characters there are of each character type:
num_clerics = curs.execute("SELECT COUNT(DISTINCT character_ptr_id) FROM charactercreator_cleric").fetchall()[0][0]
num_fighters = curs.execute("SELECT COUNT(DISTINCT character_ptr_id) FROM charactercreator_fighter").fetchall()[0][0]
num_thiefs = curs.execute("SELECT COUNT(DISTINCT character_ptr_id) FROM charactercreator_thief").fetchall()[0][0]
num_mages = curs.execute("SELECT COUNT(DISTINCT character_ptr_id) FROM charactercreator_mage").fetchall()[0][0]
num_mages_necromancers = curs.execute("SELECT COUNT(DISTINCT mage_ptr_id) FROM charactercreator_necromancer").fetchall()[0][0]

# Print answers:
print("(2) How many of each specific subclass (of Character)?")
print(f"# of clerics: {num_clerics}")
print(f"# of fighters: {num_fighters}")
print(f"# of thiefs: {num_thiefs}")
print(f"# of mages: {num_mages} (of which {num_mages_necromancers} are necromancers)")

# Check to make sure we covered all characters and type:
assert num_chars == num_clerics + num_fighters + num_thiefs + num_mages

(2) How many of each specific subclass (of Character)?
# of clerics: 75
# of fighters: 68
# of thiefs: 51
# of mages: 108 (of which 11 are necromancers)


**(3) How many total Items?**

In [110]:
# SQLite queries to find how many unique items, how many of those are weapons, and how many total items 
# held by characters there are:
num_unique_items = curs.execute("SELECT COUNT(DISTINCT item_id) FROM armory_item").fetchall()[0][0]
num_weapons = curs.execute("SELECT COUNT(DISTINCT item_ptr_id) FROM armory_weapon").fetchall()[0][0]
total_items_held = curs.execute("SELECT COUNT(item_id) FROM charactercreator_character_inventory").fetchall()[0][0]

# Print answers:
print(f"# of unique item types: {num_unique_items} items ({num_weapons} weapons, {num_unique_items - num_weapons} non-weapons)")
print(f"# of total items held by characters in the game: {total_items_held} items")

# of unique item types: 174 items (37 weapons, 137 non-weapons)
# of total items held by characters in the game: 898 items


**(4) How many of the Items are weapons? How many are not?**

See answer to Question #3 above.

**(5) How many Items does each character have? (Return first 20 rows)**

In [111]:
# SQLite query to find how many total items each character has:
query_05_results = curs.execute("SELECT character_id, COUNT(item_id) FROM charactercreator_character_inventory GROUP BY character_id ORDER BY character_id LIMIT 20").fetchall()

# Same query on multiple lines (better code style?):
# SELECT character_id, COUNT(item_id) 
# FROM charactercreator_character_inventory 
# GROUP BY character_id 
# ORDER BY character_id
# LIMIT 20

# Display results as a table by making into a pandas dataframe:
pd.DataFrame(query_05_results, columns=['character_id', 'num_items_held'])


,character_id,num_items_held
0,1,3
1,2,3
2,3,2
3,4,4
4,5,4
5,6,1
6,7,5
7,8,3
8,9,4
9,10,4


**(6) How many Weapons does each character have? (Return first 20 rows)**

In [112]:
# SQLite query to find how many weapons each character has:
query_06_results = curs.execute("SELECT inventory.character_id, COUNT(items.item_id) FROM charactercreator_character_inventory as inventory, armory_item as items, armory_weapon AS weapons WHERE inventory.item_id = items.item_id AND items.item_id = weapons.item_ptr_id GROUP BY inventory.character_id LIMIT 20").fetchall()

# Same query on multiple lines (better code style?):
# SELECT inventory.character_id, COUNT(items.item_id)
# FROM charactercreator_character_inventory as inventory, armory_item as items, armory_weapon AS weapons
# WHERE inventory.item_id = items.item_id AND items.item_id = weapons.item_ptr_id
# GROUP BY inventory.character_id
# LIMIT 20

# Display results as a table by making into a pandas dataframe:
pd.DataFrame(query_06_results, columns=['character_id', 'num_weapons_held'])

,character_id,num_weapons_held
0,5,2
1,7,1
2,11,1
3,20,1
4,22,1
5,23,1
6,26,1
7,27,3
8,29,2
9,30,1


In [113]:
# CHECK answer (BUT don't use this syntax -- BETWEEN 138 AND 174 isn't as good as directly 
# checking if item_id is in weapons):
curs.execute("SELECT character_id, COUNT(item_id) FROM charactercreator_character_inventory WHERE item_id BETWEEN 138 AND 174 GROUP BY character_id ORDER BY character_id LIMIT 20").fetchall()

# SELECT character_id, COUNT(item_id)
# FROM charactercreator_character_inventory
# WHERE item_id BETWEEN 138 AND 174
# GROUP By character_id
# ORDER BY character_id
# LIMIT 20

[(5, 2),
 (7, 1),
 (11, 1),
 (20, 1),
 (22, 1),
 (23, 1),
 (26, 1),
 (27, 3),
 (29, 2),
 (30, 1),
 (32, 1),
 (34, 1),
 (35, 2),
 (36, 3),
 (37, 2),
 (38, 2),
 (39, 2),
 (40, 1),
 (41, 1),
 (47, 1)]

**(7) On average, how many Items does each Character have?**

In [114]:
# CHECK vs. correct answers:
# Total items held: 898
# Total characters: 302
# Avg. items per character: 2.9735099337748343

# Total weapons held: 203
# Avg. weapons per character: 0.22605790645879734
# Characters holding weapons: 155
# Avg. weapons per character holding >=1 weapons: 1.3096774193548386

**(8) On average, how many Weapons does each character have?**

In [115]:
# CHECK vs. correct answers:
# Total items held: 898
# Total characters: 302
# Avg. items per character: 2.9735099337748343

# Total weapons held: 203
# Avg. weapons per character: 0.22605790645879734
# Characters holding weapons: 155
# Avg. weapons per character holding >=1 weapons: 1.3096774193548386

You do not need all the tables - in particular, the account_*, auth_*, django_*, and socialaccount_* tables are for the application and do not have the data you need. the charactercreator_* and armory_* tables and where you should focus your attention. armory_item and charactercreator_character are the main tables for Items and Characters respectively - the other tables are subsets of them by type (i.e. subclasses), connected via a key (item_id and character_id).

You can use the DB Browser or other tools to explore the data and work on your queries if you wish, but to complete the assignment you should write a file rpg_queries.py that imports sqlite3 and programmatically executes and reports results for the above queries.

Some of these queries are challenging - that's OK! You can keep working on them tomorrow as well (we'll visit loading the same data into PostgreSQL). It's also OK to figure out the results partially with a query and partially with a bit of logic or math afterwards, though doing things purely with SQL is a good goal. Subqueries and aggregation functions may be helpful for putting together more complicated queries.